# 读取excel中的半结构化数据

1. 读取信息：
    1. 读取案例表得到dataframe
    1. 读取记录表得到dataframe
    1. 整合两个dataframe
    1. 写入csv文件，作为entity.csv
1. 手动写入relation.csv
1. 存入Neo4j数据库：
    1. 读入csv文件
    1. 存入Neo4j

In [ ]:
编号 机型 机号 工况 小时数
故障内容 故障现象 可能故障原因 检查过程 检查结果及处置 故障原因 故障代码 处理心得

In [ ]:
状态 机号 任务号 报告时间 区域 服务人员 是否支援 最迟完成日期 服务日期 故障发生日期 品牌 工作小时(上报) 工作小时(监控) 工作小时之差 是否停机 故障内容 结果内容 未完成原因 是否修复 未修复原因 故障原因 故障代码 故障代码翻译后的文字 工作代码 工作代码翻译后的文字 是否有偿服务 应收修理工时费(元) 应收零件费(元) 机型 实收故障诊断工时费(元) 实收合计(元) 实收零件费(元) 实收修复工时费(元) 交机日期 工作种类 工作条件 地面条件 环境 与客户沟通结果 开始服务时间 结束服务时间 出发时间 返回时间 里程 是否在保 服务人员意见 用户满意程度 用户意见 纬度 经度 机器所在地 外协费用(元) 联系人 服务内容 
# TODO：搞如何融合两个表，做成PPT

## 初始化

In [1]:
import xlrd
import pandas as pd
import re 
import numpy as np
import os
from tqdm import tqdm


In [17]:
#自定义一个错误类型，利用它去跳出多重循环
class NextKey(Exception):
    pass

## 读取案例表

### 读取一个案例excel文件

In [18]:
# 编号,机型,机号，小时数,工况,故障现象，故障内容，可能故障原因,检查过程，故障原因，故障代码,检查结果及处置，处理心得
# 一个以上面注释为key的map，value是key对应的几种形式。
M = {'案例编号':['编号'], '机型':['机型'], '机号':['机号'], '小时数':['小时数'], '工况':['工况','作业环境'], \
        '故障现象':['机器的故障现象','故障现象'], '故障内容':['故障内容'], '可能故障原因':['可能故障原因','分析可能导致故障的原因'], \
        '检查过程':['检查过程','检查步骤'], '故障原因':['故障原因','结论'], '故障代码':['故障代码'], \
        '检查结果及处置':['检查结果及处置','维修方案'], '处理心得':['处理心得']}
    
def getInfoFromExcel(filename):
    # 打开文件
    try:
        book = xlrd.open_workbook(filename)
    except Exception as e:
        print("Error:" + str(e) + " when open_workbook of " + filename) 
        return


    df_list = []
    
    # 对于每一个sheet，进行处理，得到一行记录
    for s in book.sheets():
        # 跳过空表
        if (s.nrows == 0):
            continue
        sheet = []
        for r in range(s.nrows):
#             print("第%d行："%r)
            sheet.append(s.row_values(r))
#             print (s.row_values(r))
        
        # 建立字典，key-value为属性及其对应的值
        dict = {}
        dict['Path'] = filename
        dict['SheetCount'] = len(book.sheets())
        dict['SheetName'] = s.name
        for key_real in M.keys():
            try:
                for key_virtual in M[key_real]:
        #             print(key)
                    for i_row in range(0,len(sheet)):
                #         print(row)
        #                 print("i_row = " + str(i_row))
                        for i_element in range(0,len(sheet[i_row])):
                #             print(element)
                            # 与每个单元格进行对比
#                             if (str(key_virtual) in str(sheet[i_row][i_element])) and (i_row+1 < len(sheet)):
                            if (str(sheet[i_row][i_element]).startswith(str(key_virtual))) and (i_row+1 < len(sheet)):
                                content = str(sheet[i_row+1][i_element])
                                if (key_virtual == '结论' or key_virtual == '维修方案') and i_element+3 < len(sheet[i_row]):
                                    content = str(sheet[i_row][i_element+3])
                                # 有的单元格把key和value写在了一起，导致本单元格很长
                                if len(str(sheet[i_row][i_element])) > 2 * len(key_virtual): 
                                    content = str(sheet[i_row][i_element])
                                    content = content[len(key_virtual):]
                                    if content[0] == ':' or content[0] == '：':
                                        content = content[1:]
                                #去掉所有空白字符（包括回车空格tab）
                                content = re.sub('\s','',content) 
                                dict[key_real] = content
                                raise NextKey
            except NextKey: # 使用NextKey直接跳出多重循环
                pass
                    
        df = pd.DataFrame(dict,index=[0])
        df_list.append(df)
    df_case = df_list[0] if (len(df_list) > 0) else pd.DataFrame() # df_case，有的excel里面啥都没有
    for i in range(1,len(df_list)):
        df_case = pd.concat([df_case,df_list[i]],sort=False,ignore_index=True)
#     df.to_csv("output.csv",quoting=1,sep=';',index=False,encoding='utf-16')
    return df_case

#### 测试代码

In [7]:
string  = "检查步骤：①现场检查燃油系统及进气系统正常       ②监控面板无故障代码显示                ③测量发动机水温传感器（7861-92-3380）电阻值为13欧姆，与标准不符，以新品更换后故障消失"
substr = "检查步骤"
string.startswith(substr)

True

In [420]:
filename = "E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例1\案例\PC-7实机故障案例.xls"
book = xlrd.open_workbook(filename)
for sheet in book.sheets():
    if (sheet.nrows == 0):
        continue
    print(sheet.nrows)
print(len(book.sheets()))
getInfoFromExcel(filename)
# sheet = book.sheet_by_index(1)
# for i_row in range(0,sheet.nrows):
#     print(sheet.row_values(i_row))

17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
20


,Path,SheetCount,SheetName,机型,机号,工况,故障现象,可能故障原因,检查过程,故障原因,检查结果及处置,故障代码
0,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet1,PC200-7,DBB1664,高原、沿海、山区、河床、沙地等,车窗清洗液马达不能旋转,①清洗液马达部品不良②电器线路有故障③监控器开关面板故障,（1）现场测量清洗液马达电阻值：①和②之间为8.3Ω（标准5~20Ω），电压为24V（标准2...,洗涤器马达部品不良,更换水箱总成（21T-06-11350）,NaN
1,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet2,PC220-7,DBH0311,高原、沿海、山区、河床、沙地等,斗杆自然下降量大,①斗杆油缸内泄②斗杆主滑阀芯回位不良,（1）空载实验，动臂举生到最高位、斗杆和铲斗油缸全部收回，30分钟斗杆油缸伸出量80mm（2...,斗杆油缸内泄,更换油缸修理包（707-99-58070）,NaN
2,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet20,PC220-7,DBH0457,高原、沿海、山区、河床、沙地等,右侧终传动异响,①终传动异常磨损②终传动内部有异物③内部齿轮润滑不良④行走马达内部故障,（1）驾驶员在工作过程中听到异响后，立即现场检查终传动油位正常（2）打开终传动放油口发现油内...,终传动行星齿轮断裂,更换终传动总成1件（206-27-00421）,NaN
3,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet19,PC200-7,DBB0802,高原、沿海、山区、河床、沙地等,整机动作慢,①发动机转速低②主泵排量小③自主减压阀故障④电控系统故障,①经服务人员现场检查确认该机工作时工作装置速度明显较慢，做复合动作时工作装置几乎无法动作②监...,电磁阀部品不良,更换电磁阀总成,①经服务人员现场检查确认该机工作时工作装置速度明显较慢，做复合动作时工作装置几乎无法动作②监...
4,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet18,PC220-7,DBH0118,高原、沿海、山区、河床、沙地等,发动机在中速操作时，机器出现较大的机械震动,①发动机体的震动②主泵内部故障③液压系统回路工作不稳定引起的液压冲击,（1）经服务人员现场确认，机器震动是由液压主泵的剧烈抖动引起（2）检查液压油及液压油箱出口滤...,主泵内部出现异常磨损,更换主泵总成（708-26-00112）及液压油,NaN
5,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet17,PC200-7,DBB0954,高原、沿海、山区、河床、沙地等,工作装置连接销衬套开裂,①衬套制造缺陷②黄油润滑不良③异物进入衬套导致异常磨损,①经服务人员现场检查确认动臂与斗杆的连接销轴处异响严重，对该处加强润滑，异响仍然很响②拆卸动...,轴承衬套部品不良,更换轴承衬套1件（20Y-70-31271）,NaN
6,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet16,PC220-7,DBH0130,高原、沿海、山区、河床、沙地等,蓄电池的接线极柱上产生大量灰白色粉末,①电解液的腐蚀②蓄电池制品不良③接线松动,（1）该机使用至760H时，用户来电反映蓄电池一极柱有碱析出，服务赶往现场确认在极柱及周围有...,蓄电池部品不良,更换蓄电池1件（20Y-06-D1150）,NaN
7,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet15,PC200-7,DBB0003,高原、沿海、山区、河床、沙地等,发动机水温高,①散热器通风孔灰尘堵塞，散热不良②风扇皮带松动，散热不良③节温器打不开，冷却液小循环④水温感...,①现场检查发现机器工作后较短时间内即水温升高②检查水箱盖上的标记为“0.5S”（高原地区车辆...,未对应安装高原规格车水箱盖,更换“0.9”标记的水箱盖（20Y-03-22110）,NaN
8,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet14,PC200-7,DBB0452,步骤：①现场检查铲斗上距左侧耳板约20CM处，钢板开裂长约30CM②用户在山坡地带作业，作业...,铲斗钢板开裂,①钢板材料缺陷或焊接缺陷②用户使用不当,①现场检查铲斗上距左侧耳板约20CM处，钢板开裂长约30CM②用户在山坡地带作业，作业面有岩...,大容量铲斗不适于山区作业,更换铲斗,NaN
9,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,20,Sheet13,PC220-7,DBH0294,高原、沿海、山区、河床、沙地等,主阀处漏油,①密封件破损或失效②主阀体制造缺陷③接头或法兰紧固力矩不足,（1）该机为修建作业，每日工作约10小时，环境温度10度以上（2）现场检查发现漏油位置为主控...,LS管接头处O型圈破损,清除毛刺并更换O型圈（07002-11423）,NaN


In [401]:
df_combine = getInfoFromExcel('1斗杆速度慢.xls')
filename = r"E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例\A-0043.xls"
df_tmp = getInfoFromExcel(filename)
df_combine = pd.concat([df_combine,df_tmp],sort=False,ignore_index=True)
df_combine.replace('',np.nan,inplace=True)
print(df_combine.shape[1] - df_combine.count(axis=1))
df_combine['NaN_number'] = df_combine.shape[1] - df_combine.count(axis=1)
df_combine

0    2
1    3
2    7
dtype: int64


,Path,SheetCount,SheetName,编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,处理心得,可能故障原因,机号,NaN_number
0,1斗杆速度慢.xls,3,Sheet1,C-0002,PC360-7,113.0,土壤,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆速度慢,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,斗杆HI阀滑阀从螺纹部分脱开,P223MS,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,如无法确定问题点，就使用排除法去一步一步的检查。云南小松也有机器出现相同故障，但故障表现不同,NaN,NaN,2
1,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,3,Sheet1,A-0043,PC200-7,NaN,NaN,全车动作慢,全车动作慢,分析可能导致故障的原因：泵控制器损坏,泵控制器损坏,DABOKB,泵控制器内部接地,12号保险是给监控面板和泵控制器同时供电，一定要分清哪个部件短路造成的故障。,检查步骤：1、检查液压系统均正常(主泵出口压力350公斤/低压压力40公斤/液压油及滤芯/液...,NaN,3
2,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,3,Sheet2,NaN,PC200-7,NaN,电场清渣作业,障现象：全车动作慢,NaN,①检查液压系统均正常(主泵出口压力/低压压力/液压油及滤芯/液压管路等均正常)②检查保险盒保...,NaN,NaN,NaN,NaN,泵控制器损坏,DBB0809,7


In [501]:
# 统计文件类型
import os
rootpath = "E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例"
# rootpath = "data/"

# files = os.listdir(rootpath)
excel_num = 0
excel_num2 = 0
ppt_num = 0
ppt_num2 = 0
doc_num = 0
doc_num2 = 0
other_num = 0
for parent,dirnames,filenames in os.walk(rootpath):
#     for dirname in dirnames:
#         print("parent folder is:" + parent)
#         print("dirname is:" + dirname)
    for filename in filenames:
        filename_fullpath = os.path.join(parent,filename)
#         print(filename_fullpath)
        if filename_fullpath.endswith('.xls'):
            excel_num += 1
        elif filename_fullpath.endswith('xlsx'):
            excel_num += 1
            excel_num2 += 1
        elif filename_fullpath.endswith('.ppt'):
            ppt_num += 1 
        elif filename_fullpath.endswith('.pptx'):
            ppt_num += 1 
            ppt_num2 += 1 
        elif filename_fullpath.endswith('.doc'):
            doc_num += 1 
        elif filename_fullpath.endswith('.docx'):
            doc_num += 1 
            doc_num2 += 1 
        else:
            other_num += 1
            print(filename_fullpath)
print(excel_num, excel_num2, ppt_num, ppt_num2, doc_num, doc_num2, other_num)

E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例\故障案例集锦（新）-62.pdf
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例1\案例\挖掘机液压系统常见故障的诊断与排除.txt
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例1\案例\故障案例集锦（新）.pdf
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例1\案例\清洗液压系统方法
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例1\案例2\故障案例1\挖掘机液压系统常见故障的诊断与排除.txt
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例1\案例2\故障案例1\清洗液压系统方法
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例2\PC200以上-PC300以下\发动机\发动机转速下降
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例2\PC300及以上-PC450及以下\.DS_Store
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例2\服务案例\服务本部一线故障案例考核流程图.vsd
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_20161121\故障案例-王璐-2016.11.21\故障案例2\服务案例\河北小松12年1月案例.rar
E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\补充案例库_王璐_2016112

In [505]:
len(df_combine)

21

### 大规模处理案例excel

In [ ]:
rootpath = "E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例"
# rootpath = "data/"

# files = os.listdir(rootpath)
df_case = getInfoFromExcel('1斗杆速度慢.xls')

for parent,dirnames,filenames in os.walk(rootpath):
#     for dirname in dirnames:
#         print("parent folder is:" + parent)
#         print("dirname is:" + dirname)
    for filename in filenames:
        filename_fullpath = os.path.join(parent,filename)
#         print(filename_fullpath)
        if filename_fullpath.endswith('.xls'):
#             print(filename_fullpath)
            df_tmp = getInfoFromExcel(filename_fullpath)
#             df_tmp = df_tmp[0:1] if  else pd.DataFrame()
    
            df_case = pd.concat([df_case,df_tmp],sort=False,ignore_index=True)
#         print("parent folder is:" + parent)
#         print("filename with full path:" + os.path.join(parent, filename))
#     print(len(filenames))
df_case[':LABEL'] = '实例'

df_case.replace('',np.nan,inplace=True)
df_case['NaN_number'] = df_case.shape[1] - df_case.count(axis=1)

#去除相同案例
keys = list(M.keys())
MapUnique = {}
for index in df_case.index:
    string = ""
    for key in keys:
        string += " " + str(df_case[key][index])
    if string in MapUnique:
#         print("dumplicate bewteen these two files shown below") 
#         print(MapUnique[string])
#         print(df_case['Path'][index])
        df_case.drop(index=index,inplace=True)
    else:
        MapUnique[string] = df_case['Path'][index]

print(df_case['NaN_number'].value_counts().sort_index())
for index in df_case.index:
    if (df_case['NaN_number'][index] >= 7):
        df_case.drop(index=index, inplace=True)

#### 一些其他处理

In [25]:
# 删掉所有不需要的列，但debug时要注掉下面这句
df_case.drop(['Path', 'SheetCount','SheetName',':LABEL','NaN_number'], axis=1,inplace=True)
df_case.drop(['可能故障原因'], axis=1,inplace=True)

In [41]:
df_case.loc[1,'小时数'] = float(df_case.loc[12,'小时数'])

ValueError: could not convert string to float: '402H'

In [42]:
df_case.loc[0:10,'小时数']

0       113.0
1        9532
2     16658.0
3     12340.0
4         NaN
5         NaN
6      1136.0
7      2577.0
8      1331.0
9      4231.0
10     3523.0
Name: 小时数, dtype: object

In [82]:
s = 'helloworld小时'
print(re.match(r'\D*(\d*)\D*',s).group(1))
float(re.match(r'\D*(\d*)\D*',s).group(1))
# re.match(r'he(.*)',s).group(1)

ValueError: could not convert string to float: 

In [102]:
# 处理“小时数”那列，全部从奇怪string处理为float
for idx in tqdm(list(df_case.index)):
    try_times = 0
    while try_times >= 0:
        try:
            df_case.loc[idx,'小时数'] = float(df_case.loc[idx,'小时数'])
            try_times = -1
        except ValueError:
            print(str(idx)+ ' ' + df_case.loc[idx,'小时数'])
            s = df_case.loc[idx,'小时数']
            if s.endswith('H') or s.endswith('h'):
                df_case.loc[idx,'小时数'] = s[0:-1]
            elif s.endswith('小时'):
                df_case.loc[idx,'小时数'] = s[0:-2]
            else: # 取出中间是一段数字的部分
                df_case.loc[idx,'小时数'] = re.match(r'\D*(\d*)\D*',s).group(1)

            try_times += 1
        if (try_times > 2):
            df_case.loc[idx,'小时数'] = float('NaN')
    if df_case.loc[idx,'小时数'] < 0:
        print('wtf:',idx,df_case.loc[idx,'小时数'])
                
df_case['小时数'].unique()

  0%|                                                                                          | 0/567 [00:00<?, ?it/s]

12 402H
13 312H
17 5452H
18 13121H
19 4771H
25 259H
54 788H
55 4025H
57 2911H
62 12000H
69 564H
71 3200H
72 2614H
73 320H
74 2325H
75 1505H
76 4630H
78 5827H
84 35H
113 4500H
121 7000H
122 3188H
123 5823H
126 4620H
129 1600H
131 2450h
145 6500H
146 15200H
153 8600H
155 385H
158 6000H


 28%|█████████████████████▊                                                        | 159/567 [00:00<00:00, 1578.53it/s]

160 1730H
161 84H
165 11854H
170 17H
174 旧机不祥
174 
174 
176 6350H
177 6320H
206 2652H
210 2133H
211 1052H
217 1500H
221 5300H
230 2300H
236 1500H
237 8790H
250 1521H
255 243h
262 1974H
267 385H
268 2634H
274 13361H
281 8500H
286 1817H
288 449H
289 449H
290 2028H


 51%|████████████████████████████████████████                                      | 291/567 [00:00<00:00, 1482.23it/s]

...

410 1056H
411 4537H
414 3188H
425 661H
431 9800H
438 1259H
439 9711H
444 1593H
450 DBP2944
453 7156H
460 7095H
467 1861H
495 3305H
496 2269H
537 1732H
539 335H
540 19348H
543 23286.5h
546 2594H
549 9353小时


 97%|███████████████████████████████████████████████████████████████████████████▉  | 552/567 [00:00<00:00, 1386.17it/s]

552 11102H
556 11235H
562 DZ16155


100%|██████████████████████████████████████████████████████████████████████████████| 567/567 [00:00<00:00, 1356.74it/s]


array([113.0, 9532.0, 16658.0, 12340.0, nan, 1136.0, 2577.0, 1331.0,
       4231.0, 3523.0, 1987.0, 402.0, 312.0, 258.0, 4800.0, 9620.0,
       5452.0, 13121.0, 4771.0, 3200.0, 582.0, 7523.0, 3340.0, 259.0,
       145.0, 1860.0, 954.0, 9760.0, 4700.0, 2398.0, 700.0, 3700.0, 469.0,
       6200.0, 8650.0, 3500.0, 2700.0, 692.0, 870.0, 668.0, 6000.0,
       5500.0, 125.0, 788.0, 4025.0, 2911.0, 2180.0, 4495.0, 1328.0,
       2758.0, 12000.0, 2304.0, 3632.0, 564.0, 1787.0, 2614.0, 320.0,
       2325.0, 1505.0, 4630.0, 8926.0, 5827.0, 1026.0, 1705.0, 2663.0,
       4875.0, 35.0, 142.0, 3456.0, 6731.0, 1450.0, 1734.0, 1256.0,
       4500.0, 2000.0, 5230.0, 2500.0, 6603.0, 1780.0, 4753.0, 7000.0,
       3188.0, 5823.0, 11480.0, 9200.0, 4620.0, 100.0, 1600.0, 8300.0,
       2450.0, 1703.0, 102.0, 430.0, 13.0, 1842.0, 7350.0, 6500.0,
       15200.0, 2250.0, 5350.0, 8600.0, 2372.0, 385.0, 4560.0, 3000.0,
       3856.0, 1730.0, 84.0, 7655.0, 8160.0, 11854.0, 1652.0, 70.0,
       1745.0, 17.0, 530

In [103]:
df_case['小时数'].unique().sort()

In [105]:
# 增加“工作小时范围”列（由“小时数”离散化得到）
label_num = 15
bins = pd.IntervalIndex.from_tuples([(0, 100), (100, 1000), (1000, 5000),(5000,10000),(10000,30000),(30000,100000)],closed='left')

df_case['工作小时范围'] = pd.cut(df_case['小时数'], bins)
df_case['工作小时范围']

0         [100, 1000)
1       [5000, 10000)
2      [10000, 30000)
3      [10000, 30000)
4                 NaN
5                 NaN
6        [1000, 5000)
7        [1000, 5000)
8        [1000, 5000)
9        [1000, 5000)
10       [1000, 5000)
11       [1000, 5000)
12        [100, 1000)
13        [100, 1000)
14        [100, 1000)
15       [1000, 5000)
16      [5000, 10000)
17      [5000, 10000)
18     [10000, 30000)
19       [1000, 5000)
20                NaN
21       [1000, 5000)
22        [100, 1000)
23      [5000, 10000)
24       [1000, 5000)
25        [100, 1000)
26        [100, 1000)
27       [1000, 5000)
28        [100, 1000)
29      [5000, 10000)
            ...      
537      [1000, 5000)
538     [5000, 10000)
539       [100, 1000)
540    [10000, 30000)
541    [10000, 30000)
542    [10000, 30000)
543    [10000, 30000)
544    [10000, 30000)
545      [1000, 5000)
546      [1000, 5000)
547       [100, 1000)
548      [1000, 5000)
549     [5000, 10000)
550     [5000, 10000)
551       

#### 测试代码

In [61]:
print(df_case.shape)
df_combine = 0
print(df_case.shape)
df_case.head()

(567, 18)
(567, 18)


,Path,SheetCount,SheetName,编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,处理心得,可能故障原因,机号,:LABEL,NaN_number
0,1斗杆速度慢.xls,3,Sheet1,C-0002,PC360-7,113.0,土壤,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆速度慢,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,斗杆HI阀滑阀从螺纹部分脱开,P223MS,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,如无法确定问题点，就使用排除法去一步一步的检查。云南小松也有机器出现相同故障，但故障表现不同,NaN,NaN,实例,2
1,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,3,Sheet1,A-0001,PC220-6,9532.0,挖土,"该车在正常工作时,全车动作突然消失,",自压减压阀卡死,"1.该机使用破碎器，作业率大约为30％，累计3000小时未更换液压油,检查液压油，发现油液污...",液压油过脏,6001ZB,判断：液压油过脏．处理：①再次清洗液压油箱，用和好的面粘净油箱中的杂质．②清洗液压油散热器，...,"1、挖掘机使用破碎器时,必须按照小松保养要求进行保养（液压油和滤芯）。2、清洗液压系统时，必...",NaN,NaN,实例,2
3,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,3,Sheet1,A-0002,PC220LC-6A(原装),16658.0,挖土,发动机高怠速时，做动作发动机冒黑烟严重，甚至会使发动机熄火，同时监控器显示E02报警，进入监...,E02报警、憋车,"1.判断后泵TVC电磁阀是否正常.断开C04插件,测量后泵TVC电磁阀的阻值15欧(标准值为...",泵控制器故障,DAB0KB,判断：由以上检测结果推断泵控制器异常.处理：更换泵控制器，故障消失，一切正常。,1、后泵TVC电磁阀线圈与壳体短路，可能为泵控器内部异常原因，如没有找到故障原因，新泵控制器...,NaN,NaN,实例,2
4,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,3,Sheet1,A-0003,PC220-6,12340.0,破碎,"挖掘机安装破碎器后,试车时出现不击锤现象.",破碎器不击锤,"注:该故障出现前曾更换过备用阀块,但故障依然未解.1.当出现故障时,测量备用阀先导压力为32...",破碎器管路安装不齐全,7G91PJ,"判断:在破碎器的主控阀上(备用阀),下端连接先导油路,上端阀盖用堵堵死.开始使用破碎器时,先...","破碎器出现故障时,首先应检查破碎器安装是否正确,然后再检查系统的压力和流量等.",NaN,NaN,实例,2
5,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,1,Sheet1,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,首先分清是机械问题还是电器问题，再进一步检测，可以少走弯路。,NaN,NaN,实例,4


In [515]:
df_combine.to_csv("output.csv",quoting=1,sep=';',index=False,encoding='utf-16')
print(df_combine.shape)
df_combine

(1521, 18)


In [364]:
df_combine

,Path,SheetCount,SheetName,编号,机型,小时数,工况,故障原因,故障内容,故障代码,故障现象,检查结果及处置,处理心得,检查过程,:LABEL,NaN_number
0,1斗杆速度慢.xls,3,Sheet1,C-0002,PC360-7,113.0,土壤,斗杆HI阀滑阀从螺纹部分脱开,斗杆速度慢,P223MS,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,如无法确定问题点，就使用排除法去一步一步的检查。云南小松也有机器出现相同故障，但故障表现不同,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,实例,0
1,data/300-6不能正常调速.xls,2,保险片,NaN,PC220-7,NaN,NaN,17#保险异常,保险故障,NaN,监控面板显示不全,更换17#保险，故障消失。,PC-6型M11或13#保险虚接或断开，会使监控器指示不稳，显示部分晃动。鉴于此，监控器出现...,1、检查P01①、②端电压，24V正常；2、检查P01⑥、⑦端对地电阻0欧，正常；3、检查P...,实例,4
2,data/300-6不能正常调速.xls,2,1,B0011,PC300-6,NaN,NaN,油泵齿条卡死,NaN,A740M1,发动机不能正常调速,校泵调整后，故障消失。,齿条卡死，不能调节喷油量，从而转速不能改变。（调速拉杆和齿条为机械连接）,1、检查调速马达及电位计，正常；2、检查燃油旋钮，正常；3、启动过程中观察调速马达的动作情况...,实例,3
3,data/300-6发动机启动困难 - 副本 (2).xls,2,保险片,NaN,PC220-7,NaN,NaN,17#保险异常,保险故障,NaN,监控面板显示不全,更换17#保险，故障消失。,PC-6型M11或13#保险虚接或断开，会使监控器指示不稳，显示部分晃动。鉴于此，监控器出现...,1、检查P01①、②端电压，24V正常；2、检查P01⑥、⑦端对地电阻0欧，正常；3、检查P...,实例,4
4,data/300-6发动机启动困难 - 副本 (2).xls,2,柴油泵,test,PC300-6,NaN,NaN,柴油泵回油单向阀故障,NaN,A709AC,发动机启动困难,更换回油单向阀，故障消失。,回油单向阀损坏，有时也会有憋车现象。,"1、检查管路连接处,无松动现象;2、检查油水分离器无漏气现象；3、更换滤芯,故障依旧;4、拆...",实例,3
5,data/300-6发动机启动困难 - 副本.xls,2,保险片,NaN,PC220-7,NaN,NaN,17#保险异常,保险故障,NaN,监控面板显示不全,更换17#保险，故障消失。,PC-6型M11或13#保险虚接或断开，会使监控器指示不稳，显示部分晃动。鉴于此，监控器出现...,1、检查P01①、②端电压，24V正常；2、检查P01⑥、⑦端对地电阻0欧，正常；3、检查P...,实例,4
6,data/300-6发动机启动困难 - 副本.xls,2,柴油泵,C0010,PC300-6,NaN,NaN,柴油泵回油单向阀故障,NaN,A709AC,发动机启动困难,更换回油单向阀，故障消失。,回油单向阀损坏，有时也会有憋车现象。,"1、检查管路连接处,无松动现象;2、检查油水分离器无漏气现象；3、更换滤芯,故障依旧;4、拆...",实例,3
7,data/300-6发动机启动困难.xls,2,保险片,NaN,PC220-7,NaN,NaN,17#保险异常,保险故障,NaN,监控面板显示不全,更换17#保险，故障消失。,PC-6型M11或13#保险虚接或断开，会使监控器指示不稳，显示部分晃动。鉴于此，监控器出现...,1、检查P01①、②端电压，24V正常；2、检查P01⑥、⑦端对地电阻0欧，正常；3、检查P...,实例,4
8,data/300-6发动机启动困难.xls,2,柴油泵,C0010,PC300-6,NaN,NaN,柴油泵回油单向阀故障,NaN,A709AC,发动机启动困难,更换回油单向阀，故障消失。,回油单向阀损坏，有时也会有憋车现象。,"1、检查管路连接处,无松动现象;2、检查油水分离器无漏气现象；3、更换滤芯,故障依旧;4、拆...",实例,3


5

In [349]:
s = set()
s.add(1 )
s.add(2)
s.add(2)
32 in s 

False

In [369]:
keys = ['编号','机型','小时数','工况','故障原因','故障内容','故障代码','故障现象','检查结果及处置','处理心得','检查过程']
M = {}
for index in df_combine.index:
    string = ""
    for key in keys:
        string += " " + str(df_combine[key][index])
    if string in M:
        print("dumplicate") 
        df_combine.drop(index=index,inplace=True)
    else:
        M[string] = df_combine['Path'][index]
print(len(M))
print(M)
df_combine


5
{' C-0002 PC360-7 113.0 土壤 斗杆HI阀滑阀从螺纹部分脱开 斗杆速度慢 P223MS 复合动作时斗杆速度慢，挖掘无力。（新机，未出保） 斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。 如无法确定问题点，就使用排除法去一步一步的检查。云南小松也有机器出现相同故障，但故障表现不同 现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流压力为180kg/c㎡，斗杆PPC压力为15kg/c㎡,动臂举升PPC压力为25kg/c㎡，动臂下降PPC压力为27.5kg/c㎡，异常，其它工作装置PPC压力为29kg/c㎡，正常。在中集块靠近控制阀端将铲斗PPC油管与斗杆PPC油管调换，试车故障现象依旧，可以将PPC阀排除。将管路恢复，当先做斗杆挖掘动作然后再做动臂举升溢流动作，斗杆自动回收，此时测得斗杆PPC管路内压力为10kg/c㎡，异常。初步判断为控制阀内部串通。检查控制阀时，打开斗杆HI阀时发现，斗杆HI阀从中间螺纹处脱开。': '1斗杆速度慢.xls', ' nan PC220-7 nan nan 17#保险异常 保险故障 nan 监控面板显示不全 更换17#保险，故障消失。 PC-6型M11或13#保险虚接或断开，会使监控器指示不稳，显示部分晃动。鉴于此，监控器出现异常时，首先应检查监控器电源、接地，最后检查信号线路。 1、检查P01①、②端电压，24V正常；2、检查P01⑥、⑦端对地电阻0欧，正常；3、检查P01⑧端子电压为0V，不正常；4、检查17#保险，发现17#保险已烧坏。': 'data/300-6不能正常调速.xls', ' B0011 PC300-6 nan nan 油泵齿条卡死 nan A740M1 发动机不能正常调速 校泵调整后，故障消失。 齿条卡死，不能调节喷油量，从而转速不能改变。（调速拉杆和齿条为机械连接） 1、检查调速马达及电位计，正常；2、检查燃油旋钮，正常；3、启动过程中观察调速马达的动作情况，发现调速拉杆几乎没什么动作；4、拆开调速拉杆与喷压泵齿条连接，用手推动齿条发现很沉重。': 'data/300-6不能正常调速.xls', ' test PC300-6 nan nan 柴油泵回油单向阀故障 nan A709AC 发动机启动困难 更换回油单向阀，故障

,Path,SheetCount,SheetName,编号,机型,小时数,工况,故障原因,故障内容,故障代码,故障现象,检查结果及处置,处理心得,检查过程,:LABEL,NaN_number
0,1斗杆速度慢.xls,3,Sheet1,C-0002,PC360-7,113.0,土壤,斗杆HI阀滑阀从螺纹部分脱开,斗杆速度慢,P223MS,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,如无法确定问题点，就使用排除法去一步一步的检查。云南小松也有机器出现相同故障，但故障表现不同,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,实例,0
1,data/300-6不能正常调速.xls,2,保险片,NaN,PC220-7,NaN,NaN,17#保险异常,保险故障,NaN,监控面板显示不全,更换17#保险，故障消失。,PC-6型M11或13#保险虚接或断开，会使监控器指示不稳，显示部分晃动。鉴于此，监控器出现...,1、检查P01①、②端电压，24V正常；2、检查P01⑥、⑦端对地电阻0欧，正常；3、检查P...,实例,4
2,data/300-6不能正常调速.xls,2,1,B0011,PC300-6,NaN,NaN,油泵齿条卡死,NaN,A740M1,发动机不能正常调速,校泵调整后，故障消失。,齿条卡死，不能调节喷油量，从而转速不能改变。（调速拉杆和齿条为机械连接）,1、检查调速马达及电位计，正常；2、检查燃油旋钮，正常；3、启动过程中观察调速马达的动作情况...,实例,3
4,data/300-6发动机启动困难 - 副本 (2).xls,2,柴油泵,test,PC300-6,NaN,NaN,柴油泵回油单向阀故障,NaN,A709AC,发动机启动困难,更换回油单向阀，故障消失。,回油单向阀损坏，有时也会有憋车现象。,"1、检查管路连接处,无松动现象;2、检查油水分离器无漏气现象；3、更换滤芯,故障依旧;4、拆...",实例,3
6,data/300-6发动机启动困难 - 副本.xls,2,柴油泵,C0010,PC300-6,NaN,NaN,柴油泵回油单向阀故障,NaN,A709AC,发动机启动困难,更换回油单向阀，故障消失。,回油单向阀损坏，有时也会有憋车现象。,"1、检查管路连接处,无松动现象;2、检查油水分离器无漏气现象；3、更换滤芯,故障依旧;4、拆...",实例,3


In [517]:
print(df_combine.shape)
print("value count")
print(df_combine['NaN_number'].value_counts().sort_index())
# df_combine

(611, 18)
value count
1      15
2      98
3     180
4     149
5     114
6      17
7       4
8       5
9      15
10      7
11      4
12      2
13      1
Name: NaN_number, dtype: int64


In [70]:
print(df_combine['Path'][1131])

E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\补充案例库_王璐_20161123\案例库\PC300及以上-PC450及以下\电气\雨刷器不工作-线路短路.xls


In [48]:
df_combine['Path'][0] = ["test1","test2","test3"]
df_combine['Path'][0]
df_combine

,Path,编号,机型,工况,故障原因,故障内容,故障代码,故障现象,检查结果及处置,处理心得,检查过程,小时数,:LABEL
0,"[test1, test2, test3]",C-0002,PC360-7,土壤,斗杆HI阀滑阀从螺纹部分脱开,斗杆速度慢,P223MS,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,如无法确定问题点，就使用排除法去一步一步的检查。云南小松也有机器出现相同故障，但故障表现不同,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,NaN,实例
1,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0001,PC220-6,挖土,液压油过脏,自压减压阀卡死,6001ZB,"该车在正常工作时,全车动作突然消失,",判断：液压油过脏．处理：①再次清洗液压油箱，用和好的面粘净油箱中的杂质．②清洗液压油散热器，...,"1、挖掘机使用破碎器时,必须按照小松保养要求进行保养（液压油和滤芯）。2、清洗液压系统时，必...","1.该机使用破碎器，作业率大约为30％，累计3000小时未更换液压油,检查液压油，发现油液污...",NaN,实例
2,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0002,PC220LC-6A(原装),挖土,泵控制器故障,E02报警、憋车,DAB0KB,发动机高怠速时，做动作发动机冒黑烟严重，甚至会使发动机熄火，同时监控器显示E02报警，进入监...,判断：由以上检测结果推断泵控制器异常.处理：更换泵控制器，故障消失，一切正常。,1、后泵TVC电磁阀线圈与壳体短路，可能为泵控器内部异常原因，如没有找到故障原因，新泵控制器...,"1.判断后泵TVC电磁阀是否正常.断开C04插件,测量后泵TVC电磁阀的阻值15欧(标准值为...",NaN,实例
3,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0004,PC220-7,,空调控制面板损坏,空调系统不制冷,8700MC,空调系统不制冷，无故障代码。,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,首先分清是机械问题还是电器问题，再进一步检测，可以少走弯路。,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,,实例
4,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0005,PC220-6,,回转电磁阀阀芯受冻发卡,冬天早上不能回转,DW00M1,机器冬天温度低于零下25度，早上启动后不能回转，气温高时工作正常。,检查结果，回转锁定电磁阀阀芯有水及油腻造成阀芯发卡。处置结果：清洗回转电磁阀阀芯，故障排除。,1、环境温度对液压系统能否正常工作影响非常大，寒冷地区此问题比较突出，故障判断要考虑此因素。...,检查回转锁定电磁阀电阻正常，断开回转控制油路软管无液压油流出，,,实例
5,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0006,PC200-7,矿山,左行走吸油阀阀芯碳圈损坏,左行走吸油阀故障,L5D2FA,该车在行走时左边速度较慢，右边正常。测量压力及速度如下：P（左/前.后)=200KG/CM2...,由于该提升吸油阀（723-40-81500)的阀芯挡圈（图中第38号零件）小松不单独供应，以...,小松的压力补偿阀、吸油阀、溢流阀等阀芯上的挡圈故障较多，而小松不单独供应此类配件，在很多时候...,1、检查左行走压力补偿阀阀芯和密封——正常2、检查左行走吸油阀阀芯——阀芯正常，阀芯上的挡圈...,NaN,实例
6,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0007,PC200-7,土方,没有使用小松纯正部件,油温传感器故障,DGH2MA,该车多次更换过油温传感器，但使用一段时间后液压油温显示无油温,更换小松纯正传感器后设备使用正常。在后期的跟踪回访时设备一直正常。,1、由于用户使用非小松纯正零件，导致设备多次故障。2、非小松纯正件与监控系统不能匹配，必须使...,1、机器启动后，工作10～20分钟，检查仪表液压油温——无油温2、检查液压油温传感器插头是否...,NaN,实例
7,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0008,PC220-7,铁矿,后泵传感器故障,后泵传感器故障,DHPBMA,"该机使用到4669H时,蜂鸣器鸣响，监控器面板显示为E05",更换后泵传感器（7861-93-1650）后正常。,泵控制器传感器的5伏电源同时给压力传感器、燃油旋纽、调速马达电位计供电，三者之一一旦出现短路...,经现场检查，发动机启动后，蜂鸣器间断鸣响，用监控器调出电器系统故障记录显示为E225，DHP...,NaN,实例
8,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0009,PC200-7,矿山,伺服活塞卡死,发动机不能启动,K1A0M1,设备工作8000小时，发动机不能启动，启动时排气管有少量黑烟冒出。,由于长期没有更换液压油，引起的伺服活塞卡死、斜盘角度过大而无法启动机器。清洗液压系统更换液压...,1、在处理任何故障时，不可盲目下手，应该根据相关理论结合故障的实际情况，首先将故障原因锁定在...,1、在启动发动机时，通过仪表看到发动机转速50～60rpm，液压油压力从0一直上升到100k...,NaN,实例
9,E:\天远\知识图谱\毕设实验\天远故障卡片\故障及检修案例\故障案例_王璐_2016112...,A-0010,PC200-6,市政建设,液压油中有水混入,操作手柄重,6001BE,1、操作控制工作装置手柄时，感觉到手柄重并有反弹力;2、用户承认在深水中有过陷车情况。,1、从以上检查结果判斷操作手柄重系先导系统压力过高所致。2、处理结果;清洗液压油箱和自减压阀...,1、机器工作时水面不许超过托链轮中心线，如超过将对机器造成损害;2、如机器陷入水中进行自救时...,1、检查液压油，发现颜色异常、粘度也异常。2、检查自减压阀压力异常，87kg/cm²。标准值...,NaN,实例


#### 存表和读表

In [107]:
df_case.to_csv("df_case.csv",quoting=1,sep=';',index=False,encoding='utf-16')
print(df_case.shape)
print(df_case.columns.values)
df_case.head()

(567, 13)
['案例编号' '机型' '小时数' '工况' '故障现象' '故障内容' '检查过程' '故障原因' '故障代码' '检查结果及处置'
 '处理心得' '机号' '工作小时范围']


,案例编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,处理心得,机号,工作小时范围
0,C-0002,PC360-7,113,土壤,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆速度慢,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,斗杆HI阀滑阀从螺纹部分脱开,P223MS,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,如无法确定问题点，就使用排除法去一步一步的检查。云南小松也有机器出现相同故障，但故障表现不同,NaN,"[100, 1000)"
1,A-0001,PC220-6,9532,挖土,"该车在正常工作时,全车动作突然消失,",自压减压阀卡死,"1.该机使用破碎器，作业率大约为30％，累计3000小时未更换液压油,检查液压油，发现油液污...",液压油过脏,6001ZB,判断：液压油过脏．处理：①再次清洗液压油箱，用和好的面粘净油箱中的杂质．②清洗液压油散热器，...,"1、挖掘机使用破碎器时,必须按照小松保养要求进行保养（液压油和滤芯）。2、清洗液压系统时，必...",NaN,"[5000, 10000)"
2,A-0002,PC220LC-6A(原装),16658,挖土,发动机高怠速时，做动作发动机冒黑烟严重，甚至会使发动机熄火，同时监控器显示E02报警，进入监...,E02报警、憋车,"1.判断后泵TVC电磁阀是否正常.断开C04插件,测量后泵TVC电磁阀的阻值15欧(标准值为...",泵控制器故障,DAB0KB,判断：由以上检测结果推断泵控制器异常.处理：更换泵控制器，故障消失，一切正常。,1、后泵TVC电磁阀线圈与壳体短路，可能为泵控器内部异常原因，如没有找到故障原因，新泵控制器...,NaN,"[10000, 30000)"
3,A-0003,PC220-6,12340,破碎,"挖掘机安装破碎器后,试车时出现不击锤现象.",破碎器不击锤,"注:该故障出现前曾更换过备用阀块,但故障依然未解.1.当出现故障时,测量备用阀先导压力为32...",破碎器管路安装不齐全,7G91PJ,"判断:在破碎器的主控阀上(备用阀),下端连接先导油路,上端阀盖用堵堵死.开始使用破碎器时,先...","破碎器出现故障时,首先应检查破碎器安装是否正确,然后再检查系统的压力和流量等.",NaN,"[10000, 30000)"
4,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,首先分清是机械问题还是电器问题，再进一步检测，可以少走弯路。,NaN,NaN


In [101]:
df_case = pd.read_csv('df_case.csv',sep=';',encoding='utf-16')
print(df_case.shape)
print(df_case.columns.values)
df_case.head()

(567, 12)
['案例编号' '机型' '小时数' '工况' '故障现象' '故障内容' '检查过程' '故障原因' '故障代码' '检查结果及处置'
 '处理心得' '机号']


,案例编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,处理心得,机号
0,C-0002,PC360-7,113.0,土壤,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆速度慢,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,斗杆HI阀滑阀从螺纹部分脱开,P223MS,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,如无法确定问题点，就使用排除法去一步一步的检查。云南小松也有机器出现相同故障，但故障表现不同,NaN
1,A-0001,PC220-6,9532.0,挖土,"该车在正常工作时,全车动作突然消失,",自压减压阀卡死,"1.该机使用破碎器，作业率大约为30％，累计3000小时未更换液压油,检查液压油，发现油液污...",液压油过脏,6001ZB,判断：液压油过脏．处理：①再次清洗液压油箱，用和好的面粘净油箱中的杂质．②清洗液压油散热器，...,"1、挖掘机使用破碎器时,必须按照小松保养要求进行保养（液压油和滤芯）。2、清洗液压系统时，必...",NaN
2,A-0002,PC220LC-6A(原装),16658.0,挖土,发动机高怠速时，做动作发动机冒黑烟严重，甚至会使发动机熄火，同时监控器显示E02报警，进入监...,E02报警、憋车,"1.判断后泵TVC电磁阀是否正常.断开C04插件,测量后泵TVC电磁阀的阻值15欧(标准值为...",泵控制器故障,DAB0KB,判断：由以上检测结果推断泵控制器异常.处理：更换泵控制器，故障消失，一切正常。,1、后泵TVC电磁阀线圈与壳体短路，可能为泵控器内部异常原因，如没有找到故障原因，新泵控制器...,NaN
3,A-0003,PC220-6,12340.0,破碎,"挖掘机安装破碎器后,试车时出现不击锤现象.",破碎器不击锤,"注:该故障出现前曾更换过备用阀块,但故障依然未解.1.当出现故障时,测量备用阀先导压力为32...",破碎器管路安装不齐全,7G91PJ,"判断:在破碎器的主控阀上(备用阀),下端连接先导油路,上端阀盖用堵堵死.开始使用破碎器时,先...","破碎器出现故障时,首先应检查破碎器安装是否正确,然后再检查系统的压力和流量等.",NaN
4,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,首先分清是机械问题还是电器问题，再进一步检测，可以少走弯路。,NaN


## 读取记录表

#### 读取源文件并进行处理

In [10]:
# Note：在运行本段代码前，需要手动用EXCEL软件把csv另存为xls再另存为csv（原csv文件有某些csv文件错误）
# 源文件有两行只有第一个字段有值且错误，手动去除那两行
df_record = pd.read_csv('service-info.csv')
print(df_record.shape)
print(df_record.columns.values)
# df_record.rename({'工作小时(上报)':'小时数'},axis='columns',inplace = True)
# print(df_record.columns.values)
df_record.head()

(5173, 53)
['状态' '机号' '任务号' '报告时间' '区域' '服务人员' '是否支援' '最迟完成日期' '服务日期' '故障发生日期' '品牌'
 '工作小时(上报)' '工作小时(监控)' '是否停机' '故障内容' '结果内容' '未完成原因' '是否修复' '未修复原因' '故障原因'
 '故障代码' '故障代码翻译后的文字' '工作代码' '工作代码翻译后的文字' '是否有偿服务' '应收修理工时费(元)' '应收零件费(元)'
 '机型' '实收故障诊断工时费(元)' '实收合计(元)' '实收零件费(元)' '实收修复工时费(元)' '交机日期' '工作种类'
 '工作条件' '地面条件' '环境' '与客户沟通结果' '开始服务时间' '结束服务时间' '出发时间' '返回时间' '里程' '是否在保'
 '服务人员意见' '用户满意程度' '用户意见' '纬度' '经度' '机器所在地' '外协费用(元)' '联系人' '服务内容']


,状态,机号,任务号,报告时间,区域,服务人员,是否支援,最迟完成日期,服务日期,故障发生日期,...,是否在保,服务人员意见,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人,服务内容
0,有效,D1078,RW20150710289,2015/7/9 7:33,服务-唐山一区,赵雪,【派工】,2015/7/8,2015/7/8,2015/7/8,...,否,NaN,满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆卸涡轮增压器，分解后拆机油回油润滑管
1,有效,D1078,RW20150710289,2015/7/9 8:04,服务-唐山四区,王征,【支援派工】,2015/7/8,2015/7/8,2015/7/8,...,否,NaN,非常满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆涡轮增压器，护板，油管
2,有效,D1078,RW20150810093,2015/8/5 21:29,服务-唐山一区,赵雪,【支援派工】,2015/8/5,2015/8/4,2015/8/4,...,否,NaN,满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,检测回转卡滞报警，高压线束，发电机，变频器，蓄电器等
3,有效,D1078,RW20150810093,2015/8/5 21:35,服务-唐山四区,王征,【派工】,2015/8/5,2015/8/4,2015/8/4,...,否,NaN,非常满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,测量高压共通线路
4,有效,D1078,RW20150910676,2015/9/27 8:06,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,否,NaN,满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆卸回转马达总成，分解回转机构


#### 利用同义词表修改列名

In [109]:
rename_dict = {'工作种类':'工况','工作小时(监控)':'小时数','服务内容':'检查结果及处置'}
df_record.rename(rename_dict,axis='columns',inplace=True)
print(df_record.columns.values)
# df.rename({'工作小时(上报)':'小时数'},axis='columns',inplace = True)


['状态' '机号' '任务号' '报告时间' '区域' '服务人员' '是否支援' '最迟完成日期' '服务日期' '故障发生日期' '品牌'
 '工作小时(上报)' '小时数' '是否停机' '故障内容' '结果内容' '未完成原因' '是否修复' '未修复原因' '故障原因'
 '故障代码' '故障代码翻译后的文字' '工作代码' '工作代码翻译后的文字' '是否有偿服务' '应收修理工时费(元)' '应收零件费(元)'
 '机型' '实收故障诊断工时费(元)' '实收合计(元)' '实收零件费(元)' '实收修复工时费(元)' '交机日期' '工况' '工作条件'
 '地面条件' '环境' '与客户沟通结果' '开始服务时间' '结束服务时间' '出发时间' '返回时间' '里程' '是否在保'
 '服务人员意见' '用户满意程度' '用户意见' '纬度' '经度' '机器所在地' '外协费用(元)' '联系人' '检查结果及处置'
 '记录编号' '机器所在地址']


#### 增加“记录编号”列（记录编号=[任务号+时间顺序号]格式，其中时间顺序号是按照“报告时间”顺序的同一任务号下的编号）

In [11]:
df_record['记录编号'] = df_record['任务号']
df_record

,状态,机号,任务号,报告时间,区域,服务人员,是否支援,最迟完成日期,服务日期,故障发生日期,...,服务人员意见,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人,服务内容,记录编号
0,有效,D1078,RW20150710289,2015/7/9 7:33,服务-唐山一区,赵雪,【派工】,2015/7/8,2015/7/8,2015/7/8,...,NaN,满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆卸涡轮增压器，分解后拆机油回油润滑管,RW20150710289
1,有效,D1078,RW20150710289,2015/7/9 8:04,服务-唐山四区,王征,【支援派工】,2015/7/8,2015/7/8,2015/7/8,...,NaN,非常满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆涡轮增压器，护板，油管,RW20150710289
2,有效,D1078,RW20150810093,2015/8/5 21:29,服务-唐山一区,赵雪,【支援派工】,2015/8/5,2015/8/4,2015/8/4,...,NaN,满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,检测回转卡滞报警，高压线束，发电机，变频器，蓄电器等,RW20150810093
3,有效,D1078,RW20150810093,2015/8/5 21:35,服务-唐山四区,王征,【派工】,2015/8/5,2015/8/4,2015/8/4,...,NaN,非常满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,测量高压共通线路,RW20150810093
4,有效,D1078,RW20150910676,2015/9/27 8:06,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆卸回转马达总成，分解回转机构,RW20150910676
5,有效,D1078,RW20150910677,2015/9/27 8:14,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆装回转润滑泵总成，分解更换密封。,RW20150910677
6,有效,D1078,RW20150910677,2015/9/27 8:18,服务-唐山二区,赵本帅,【派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,非常满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,更换回转电机润滑泵密封,RW20150910677
7,有效,D1078,RW20150910676,2015/9/27 8:22,服务-唐山二区,赵本帅,【派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,非常满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆回转减速机构,RW20150910676
8,有效,D1078,RW20151010012,2015/10/8 13:14,服务-迁安二区,赵广立,【派工】,2015/10/8,2015/10/2,2015/10/2,...,NaN,满意,NaN,39.146099,118.453533,河北唐山市滦南县附近,0.0,李洪飞,组装回转马达,RW20151010012
9,有效,D1078,RW20151010013,2015/10/8 13:25,服务-迁安二区,赵广立,【派工】,2015/10/8,2015/10/2,2015/10/2,...,NaN,满意,NaN,39.146099,118.453533,河北唐山市滦南县附近,0.0,李洪飞,安装回转马达,RW20151010013


In [12]:
task_map = {}
for idx in tqdm(list(df_record.index)):
    task_id = df_record.loc[idx,'任务号']
    if task_id in task_map.keys():
        task_map[task_id] += 1
    else:
        task_map[task_id] = 1
    df_record.loc[idx,'记录编号'] = df_record.loc[idx,'任务号'] + str("%03d"%task_map[task_id])
df_record

100%|████████████████████████████████████████████████████████████████████████████| 5173/5173 [00:02<00:00, 2205.28it/s]


,状态,机号,任务号,报告时间,区域,服务人员,是否支援,最迟完成日期,服务日期,故障发生日期,...,服务人员意见,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人,服务内容,记录编号
0,有效,D1078,RW20150710289,2015/7/9 7:33,服务-唐山一区,赵雪,【派工】,2015/7/8,2015/7/8,2015/7/8,...,NaN,满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆卸涡轮增压器，分解后拆机油回油润滑管,RW20150710289001
1,有效,D1078,RW20150710289,2015/7/9 8:04,服务-唐山四区,王征,【支援派工】,2015/7/8,2015/7/8,2015/7/8,...,NaN,非常满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆涡轮增压器，护板，油管,RW20150710289002
2,有效,D1078,RW20150810093,2015/8/5 21:29,服务-唐山一区,赵雪,【支援派工】,2015/8/5,2015/8/4,2015/8/4,...,NaN,满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,检测回转卡滞报警，高压线束，发电机，变频器，蓄电器等,RW20150810093001
3,有效,D1078,RW20150810093,2015/8/5 21:35,服务-唐山四区,王征,【派工】,2015/8/5,2015/8/4,2015/8/4,...,NaN,非常满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,测量高压共通线路,RW20150810093002
4,有效,D1078,RW20150910676,2015/9/27 8:06,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆卸回转马达总成，分解回转机构,RW20150910676001
5,有效,D1078,RW20150910677,2015/9/27 8:14,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆装回转润滑泵总成，分解更换密封。,RW20150910677001
6,有效,D1078,RW20150910677,2015/9/27 8:18,服务-唐山二区,赵本帅,【派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,非常满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,更换回转电机润滑泵密封,RW20150910677002
7,有效,D1078,RW20150910676,2015/9/27 8:22,服务-唐山二区,赵本帅,【派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,非常满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆回转减速机构,RW20150910676002
8,有效,D1078,RW20151010012,2015/10/8 13:14,服务-迁安二区,赵广立,【派工】,2015/10/8,2015/10/2,2015/10/2,...,NaN,满意,NaN,39.146099,118.453533,河北唐山市滦南县附近,0.0,李洪飞,组装回转马达,RW20151010012001
9,有效,D1078,RW20151010013,2015/10/8 13:25,服务-迁安二区,赵广立,【派工】,2015/10/8,2015/10/2,2015/10/2,...,NaN,满意,NaN,39.146099,118.453533,河北唐山市滦南县附近,0.0,李洪飞,安装回转马达,RW20151010013001


#### 增加“机器所在地址”列（机器所在地址=(经度,纬度)）

In [13]:
df_record['机器所在地址'] = 'tmp-string'

In [14]:
for idx in tqdm(list(df_record.index)):
#     df_record['机器所在地址'][idx] = '(' + str(df_record['经度'][idx]) + ',' + str(df_record['纬度'][idx]) + ')'
    df_record.loc[idx,'机器所在地址'] = '(' + str(df_record.loc[idx,'经度']) + ',' + str(df_record.loc[idx,'纬度']) + ')'
df_record

100%|████████████████████████████████████████████████████████████████████████████| 5173/5173 [00:02<00:00, 2111.01it/s]


,状态,机号,任务号,报告时间,区域,服务人员,是否支援,最迟完成日期,服务日期,故障发生日期,...,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人,服务内容,记录编号,机器所在地址
0,有效,D1078,RW20150710289,2015/7/9 7:33,服务-唐山一区,赵雪,【派工】,2015/7/8,2015/7/8,2015/7/8,...,满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆卸涡轮增压器，分解后拆机油回油润滑管,RW20150710289001,"(118.452666,39.143599)"
1,有效,D1078,RW20150710289,2015/7/9 8:04,服务-唐山四区,王征,【支援派工】,2015/7/8,2015/7/8,2015/7/8,...,非常满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆涡轮增压器，护板，油管,RW20150710289002,"(118.452666,39.143599)"
2,有效,D1078,RW20150810093,2015/8/5 21:29,服务-唐山一区,赵雪,【支援派工】,2015/8/5,2015/8/4,2015/8/4,...,满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,检测回转卡滞报警，高压线束，发电机，变频器，蓄电器等,RW20150810093001,"(118.45313300000001,39.145)"
3,有效,D1078,RW20150810093,2015/8/5 21:35,服务-唐山四区,王征,【派工】,2015/8/5,2015/8/4,2015/8/4,...,非常满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,测量高压共通线路,RW20150810093002,"(118.45313300000001,39.145)"
4,有效,D1078,RW20150910676,2015/9/27 8:06,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆卸回转马达总成，分解回转机构,RW20150910676001,"(118.457299,39.153932)"
5,有效,D1078,RW20150910677,2015/9/27 8:14,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆装回转润滑泵总成，分解更换密封。,RW20150910677001,"(118.457299,39.153932)"
6,有效,D1078,RW20150910677,2015/9/27 8:18,服务-唐山二区,赵本帅,【派工】,2015/9/26,2015/9/26,2015/9/26,...,非常满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,更换回转电机润滑泵密封,RW20150910677002,"(118.457299,39.153932)"
7,有效,D1078,RW20150910676,2015/9/27 8:22,服务-唐山二区,赵本帅,【派工】,2015/9/26,2015/9/26,2015/9/26,...,非常满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆回转减速机构,RW20150910676002,"(118.457299,39.153932)"
8,有效,D1078,RW20151010012,2015/10/8 13:14,服务-迁安二区,赵广立,【派工】,2015/10/8,2015/10/2,2015/10/2,...,满意,NaN,39.146099,118.453533,河北唐山市滦南县附近,0.0,李洪飞,组装回转马达,RW20151010012001,"(118.45353300000001,39.146099)"
9,有效,D1078,RW20151010013,2015/10/8 13:25,服务-迁安二区,赵广立,【派工】,2015/10/8,2015/10/2,2015/10/2,...,满意,NaN,39.146099,118.453533,河北唐山市滦南县附近,0.0,李洪飞,安装回转马达,RW20151010013001,"(118.45353300000001,39.146099)"


In [110]:
#### 增加“工作小时范围”列（由“小时数”离散化得到）
bins = pd.IntervalIndex.from_tuples([(0, 100), (100, 1000), (1000, 5000),(5000,10000),(10000,30000),(30000,100000)],closed='left')
df_record['工作小时范围'] = pd.cut(df_record['小时数'],bins)
df_record['工作小时范围']

0        [5000, 10000)
1        [5000, 10000)
2        [5000, 10000)
3        [5000, 10000)
4        [5000, 10000)
5        [5000, 10000)
6        [5000, 10000)
7        [5000, 10000)
8        [5000, 10000)
9        [5000, 10000)
10       [5000, 10000)
11       [5000, 10000)
12       [5000, 10000)
13       [5000, 10000)
14       [5000, 10000)
15       [5000, 10000)
16       [5000, 10000)
17       [5000, 10000)
18       [5000, 10000)
19       [5000, 10000)
20       [5000, 10000)
21       [5000, 10000)
22       [5000, 10000)
23       [5000, 10000)
24       [5000, 10000)
25       [5000, 10000)
26       [5000, 10000)
27       [5000, 10000)
28       [5000, 10000)
29       [5000, 10000)
             ...      
5143    [10000, 30000)
5144    [10000, 30000)
5145    [10000, 30000)
5146    [10000, 30000)
5147    [10000, 30000)
5148    [10000, 30000)
5149    [10000, 30000)
5150    [10000, 30000)
5151    [10000, 30000)
5152    [10000, 30000)
5153    [10000, 30000)
5154    [10000, 30000)
5155    [10

#### 存表和读表

In [111]:
df_record.to_csv("df_record.csv",quoting=1,sep=';',index=False,encoding='utf-16')
print(df_record.shape)
print(df_record.columns.values)
df_record.head()

(5173, 56)
['状态' '机号' '任务号' '报告时间' '区域' '服务人员' '是否支援' '最迟完成日期' '服务日期' '故障发生日期' '品牌'
 '工作小时(上报)' '小时数' '是否停机' '故障内容' '结果内容' '未完成原因' '是否修复' '未修复原因' '故障原因'
 '故障代码' '故障代码翻译后的文字' '工作代码' '工作代码翻译后的文字' '是否有偿服务' '应收修理工时费(元)' '应收零件费(元)'
 '机型' '实收故障诊断工时费(元)' '实收合计(元)' '实收零件费(元)' '实收修复工时费(元)' '交机日期' '工况' '工作条件'
 '地面条件' '环境' '与客户沟通结果' '开始服务时间' '结束服务时间' '出发时间' '返回时间' '里程' '是否在保'
 '服务人员意见' '用户满意程度' '用户意见' '纬度' '经度' '机器所在地' '外协费用(元)' '联系人' '检查结果及处置'
 '记录编号' '机器所在地址' '工作小时范围']


,状态,机号,任务号,报告时间,区域,服务人员,是否支援,最迟完成日期,服务日期,故障发生日期,...,用户意见,纬度,经度,机器所在地,外协费用(元),联系人,检查结果及处置,记录编号,机器所在地址,工作小时范围
0,有效,D1078,RW20150710289,2015/7/9 7:33,服务-唐山一区,赵雪,【派工】,2015/7/8,2015/7/8,2015/7/8,...,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆卸涡轮增压器，分解后拆机油回油润滑管,RW20150710289001,"(118.452666,39.143599)","[5000, 10000)"
1,有效,D1078,RW20150710289,2015/7/9 8:04,服务-唐山四区,王征,【支援派工】,2015/7/8,2015/7/8,2015/7/8,...,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆涡轮增压器，护板，油管,RW20150710289002,"(118.452666,39.143599)","[5000, 10000)"
2,有效,D1078,RW20150810093,2015/8/5 21:29,服务-唐山一区,赵雪,【支援派工】,2015/8/5,2015/8/4,2015/8/4,...,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,检测回转卡滞报警，高压线束，发电机，变频器，蓄电器等,RW20150810093001,"(118.45313300000001,39.145)","[5000, 10000)"
3,有效,D1078,RW20150810093,2015/8/5 21:35,服务-唐山四区,王征,【派工】,2015/8/5,2015/8/4,2015/8/4,...,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,测量高压共通线路,RW20150810093002,"(118.45313300000001,39.145)","[5000, 10000)"
4,有效,D1078,RW20150910676,2015/9/27 8:06,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆卸回转马达总成，分解回转机构,RW20150910676001,"(118.457299,39.153932)","[5000, 10000)"


In [108]:
df_record = pd.read_csv('df_record.csv',sep=';',encoding='utf-16')
print(df_record.shape)
print(df_record.columns.values)
df_record.head()

(5173, 55)
['状态' '机号' '任务号' '报告时间' '区域' '服务人员' '是否支援' '最迟完成日期' '服务日期' '故障发生日期' '品牌'
 '工作小时(上报)' '工作小时(监控)' '是否停机' '故障内容' '结果内容' '未完成原因' '是否修复' '未修复原因' '故障原因'
 '故障代码' '故障代码翻译后的文字' '工作代码' '工作代码翻译后的文字' '是否有偿服务' '应收修理工时费(元)' '应收零件费(元)'
 '机型' '实收故障诊断工时费(元)' '实收合计(元)' '实收零件费(元)' '实收修复工时费(元)' '交机日期' '工作种类'
 '工作条件' '地面条件' '环境' '与客户沟通结果' '开始服务时间' '结束服务时间' '出发时间' '返回时间' '里程' '是否在保'
 '服务人员意见' '用户满意程度' '用户意见' '纬度' '经度' '机器所在地' '外协费用(元)' '联系人' '服务内容' '记录编号'
 '机器所在地址']


,状态,机号,任务号,报告时间,区域,服务人员,是否支援,最迟完成日期,服务日期,故障发生日期,...,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人,服务内容,记录编号,机器所在地址
0,有效,D1078,RW20150710289,2015/7/9 7:33,服务-唐山一区,赵雪,【派工】,2015/7/8,2015/7/8,2015/7/8,...,满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆卸涡轮增压器，分解后拆机油回油润滑管,RW20150710289001,"(118.452666,39.143599)"
1,有效,D1078,RW20150710289,2015/7/9 8:04,服务-唐山四区,王征,【支援派工】,2015/7/8,2015/7/8,2015/7/8,...,非常满意,NaN,39.143599,118.452666,河北唐山市滦南县附近,0.0,李洪飞,拆涡轮增压器，护板，油管,RW20150710289002,"(118.452666,39.143599)"
2,有效,D1078,RW20150810093,2015/8/5 21:29,服务-唐山一区,赵雪,【支援派工】,2015/8/5,2015/8/4,2015/8/4,...,满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,检测回转卡滞报警，高压线束，发电机，变频器，蓄电器等,RW20150810093001,"(118.45313300000001,39.145)"
3,有效,D1078,RW20150810093,2015/8/5 21:35,服务-唐山四区,王征,【派工】,2015/8/5,2015/8/4,2015/8/4,...,非常满意,NaN,39.145000,118.453133,河北唐山市滦南县附近,0.0,李洪飞,测量高压共通线路,RW20150810093002,"(118.45313300000001,39.145)"
4,有效,D1078,RW20150910676,2015/9/27 8:06,服务-唐山一区,赵雪,【支援派工】,2015/9/26,2015/9/26,2015/9/26,...,满意,NaN,39.153932,118.457299,河北唐山市滦南县附近,0.0,李洪飞,拆卸回转马达总成，分解回转机构,RW20150910676001,"(118.457299,39.153932)"


### 测试代码

In [63]:
df_record['工作条件'].unique()

array(['装载', '挖掘', '推土或平地', '其他', '松土', '提升', nan, '运输', '挖掘1', '反铲作业',
       '卷场', '坡地铲运', '用推板刮削', '推拉', '整形'], dtype=object)

In [64]:
df_record['地面条件'].unique()

array(['沙子和石砾', '易碎岩石', '土壤', '坚硬岩石', '其他', '煤', '铺筑路', '湿地', '未铺筑平路',
       nan, '未铺筑不平路面'], dtype=object)

In [65]:
df_record['工作种类'].unique()

array(['沙子&碎石', '采石场', '土地建设(小型)', '其他种类矿', '道路建设(地方)', '公路或铁路建设', '租赁',
       '拆除', '其他', '铁路', '煤矿', '土地建设（大型）', '河流渠道及灌溉', '熔渣处理', '石灰矿',
       '隧道建设', '垃圾处理', '高速公路建设', '采石场物料搬运', '建筑物建设', '土地开发', nan, '水坝建设',
       '水泥产品', '海运', '海洋保护', '供水及水处理', '农田', '农田土地发展', '管道&线路铺设', '未知',
       '机场建设', '道路建设（全国）'], dtype=object)

In [66]:
df_case['工况'].unique()

array(['土壤', '挖土', '破碎', nan, '矿山', '土方', '铁矿', '市政建设', '道路建设', '挖掘矿粉',
       '土地开发', '市政', '石材矿采石', '硷厂角料外运', '河边修路', '水利建设', '土地建设', '采石场',
       '泥土', '铁矿开采', '采石厂', '拆迁工程', '碎石挖掘', '矿山开采', '公路建设', '电场清渣作业',
       '高原、道路建设', '公路或铁路建设', '石料厂破碎岩石', '遂道作业', '易碎岩石', '挖掘作业', '公路作业',
       '修路', '开荒种植', '园林开荒', '道路施工', '高原、山区、煤矿等', '高原、山区、沙地等。', '矿石开采',
       '石矿开采', '道路挖掘', '房建工地', '黏土挖掘', '房屋拆迁', '挖掘', '船舱推煤', '土石方',
       '碎石装载', '土石方工程', '碎石铲运', '油田装料作业', '隧洞作业', '平整场地', '物料装载', '沙石',
       '水电站建设', '料场', '土方/破碎', '石场散料装卸', '土方工程', '土石方开挖', '土方挖掘', '石料场',
       '土石施工', '岩石挖掘装车', '黏土', '拆迁', '土石挖掘', '煤矿煤炭挖掘', '沙土', '土壤挖掘',
       '铁路基础建设', '沙石、泥土', '道路', '岩石', '矿山作业', '破碎器拆迁', '河堤沙石', '公路地基',
       '劣质、斗杆选用不当', '土石方施工', '沙漠', '河沙装车', '土方作业', '挖掘、破碎', '铁路建设', '隧道',
       '破碎作业', '土石方作业', '高速公路建设', '采矿', '泥地', '隧道施工', '矿山挖掘', '沙子和石砾',
       '建筑物建设', '乡村路修建', '绿化建设', '煤矿', '装载', '水中', '土方工程等', '土方改造',
       '城市建设', '建筑施工', '建筑', '石子', '石场开采', '土石', '石场', '土、石方', '路基建设',
      

## 整合两个表格

In [112]:
df_record = pd.read_csv('df_record.csv',sep=';',encoding='utf-16')
df_case = pd.read_csv('df_case.csv',sep=';',encoding='utf-16')

In [113]:
print(df_case.shape)
print(df_case.columns.values)
print(df_record.shape)
print(df_record.columns.values)

(567, 13)
['案例编号' '机型' '小时数' '工况' '故障现象' '故障内容' '检查过程' '故障原因' '故障代码' '检查结果及处置'
 '处理心得' '机号' '工作小时范围']
(5173, 56)
['状态' '机号' '任务号' '报告时间' '区域' '服务人员' '是否支援' '最迟完成日期' '服务日期' '故障发生日期' '品牌'
 '工作小时(上报)' '小时数' '是否停机' '故障内容' '结果内容' '未完成原因' '是否修复' '未修复原因' '故障原因'
 '故障代码' '故障代码翻译后的文字' '工作代码' '工作代码翻译后的文字' '是否有偿服务' '应收修理工时费(元)' '应收零件费(元)'
 '机型' '实收故障诊断工时费(元)' '实收合计(元)' '实收零件费(元)' '实收修复工时费(元)' '交机日期' '工况' '工作条件'
 '地面条件' '环境' '与客户沟通结果' '开始服务时间' '结束服务时间' '出发时间' '返回时间' '里程' '是否在保'
 '服务人员意见' '用户满意程度' '用户意见' '纬度' '经度' '机器所在地' '外协费用(元)' '联系人' '检查结果及处置'
 '记录编号' '机器所在地址' '工作小时范围']


In [114]:
# uselessKey = ['就的','好的']
uselessKey = ['状态','报告时间','是否支援','最迟完成日期','服务日期','故障发生日期','品牌','工作小时(上报)','是否停机','结果内容','未完成原因','是否修复','未修复原因','故障代码翻译后的文字','工作代码','工作代码翻译后的文字','是否有偿服务','应收修理工时费(元)','应收零件费(元)','实收故障诊断工时费(元)','实收合计(元)','实收零件费(元)','实收修复工时费(元)','交机日期','工作条件','地面条件','环境','与客户沟通结果','开始服务时间','结束服务时间','出发时间','返回时间','里程','是否在保','服务人员意见','用户满意程度','用户意见','纬度','经度','机器所在地','外协费用(元)']
print(len(uselessKey))
uselessKey
# '好的' in uselessKey

41


['状态',
 '报告时间',
 '是否支援',
 '最迟完成日期',
 '服务日期',
 '故障发生日期',
 '品牌',
 '工作小时(上报)',
 '是否停机',
 '结果内容',
 '未完成原因',
 '是否修复',
 '未修复原因',
 '故障代码翻译后的文字',
 '工作代码',
 '工作代码翻译后的文字',
 '是否有偿服务',
 '应收修理工时费(元)',
 '应收零件费(元)',
 '实收故障诊断工时费(元)',
 '实收合计(元)',
 '实收零件费(元)',
 '实收修复工时费(元)',
 '交机日期',
 '工作条件',
 '地面条件',
 '环境',
 '与客户沟通结果',
 '开始服务时间',
 '结束服务时间',
 '出发时间',
 '返回时间',
 '里程',
 '是否在保',
 '服务人员意见',
 '用户满意程度',
 '用户意见',
 '纬度',
 '经度',
 '机器所在地',
 '外协费用(元)']

In [116]:
goodKey = []
for key in df_record.columns.values:
    if key not in uselessKey:
        goodKey.append(key)
print(len(goodKey))
goodKey
    

15


['机号',
 '任务号',
 '区域',
 '服务人员',
 '小时数',
 '故障内容',
 '故障原因',
 '故障代码',
 '机型',
 '工况',
 '联系人',
 '检查结果及处置',
 '记录编号',
 '机器所在地址',
 '工作小时范围']

In [117]:
for key in goodKey:
    print("记录编号，包含，"+key+"，label：实例关系")

记录编号，包含，机号，label：实例关系
记录编号，包含，任务号，label：实例关系
记录编号，包含，区域，label：实例关系
记录编号，包含，服务人员，label：实例关系
记录编号，包含，小时数，label：实例关系
记录编号，包含，故障内容，label：实例关系
记录编号，包含，故障原因，label：实例关系
记录编号，包含，故障代码，label：实例关系
记录编号，包含，机型，label：实例关系
记录编号，包含，工况，label：实例关系
记录编号，包含，联系人，label：实例关系
记录编号，包含，检查结果及处置，label：实例关系
记录编号，包含，记录编号，label：实例关系
记录编号，包含，机器所在地址，label：实例关系
记录编号，包含，工作小时范围，label：实例关系


In [31]:
print(len(df_case.columns.values))
for col_name in df_case.columns.values:
    print("案例编号,包含,"+col_name+",label:实例关系")

13
案例编号,包含,案例编号,label:实例关系
案例编号,包含,机型,label:实例关系
案例编号,包含,小时数,label:实例关系
案例编号,包含,工况,label:实例关系
案例编号,包含,故障现象,label:实例关系
案例编号,包含,故障内容,label:实例关系
案例编号,包含,检查过程,label:实例关系
案例编号,包含,故障原因,label:实例关系
案例编号,包含,故障代码,label:实例关系
案例编号,包含,检查结果及处置,label:实例关系
案例编号,包含,处理心得,label:实例关系
案例编号,包含,可能故障原因,label:实例关系
案例编号,包含,机号,label:实例关系


In [119]:
df_combine = pd.concat([df_case,df_record],sort=False,ignore_index=True)
df_combine.to_csv("entities.csv",quoting=1,sep=';',index=False,encoding='utf-16')
print(df_combine.shape)
print(df_combine.columns.values)
df_combine.head()

(5740, 60)
['案例编号' '机型' '小时数' '工况' '故障现象' '故障内容' '检查过程' '故障原因' '故障代码' '检查结果及处置'
 '处理心得' '机号' '工作小时范围' '状态' '任务号' '报告时间' '区域' '服务人员' '是否支援' '最迟完成日期' '服务日期'
 '故障发生日期' '品牌' '工作小时(上报)' '是否停机' '结果内容' '未完成原因' '是否修复' '未修复原因'
 '故障代码翻译后的文字' '工作代码' '工作代码翻译后的文字' '是否有偿服务' '应收修理工时费(元)' '应收零件费(元)'
 '实收故障诊断工时费(元)' '实收合计(元)' '实收零件费(元)' '实收修复工时费(元)' '交机日期' '工作条件' '地面条件'
 '环境' '与客户沟通结果' '开始服务时间' '结束服务时间' '出发时间' '返回时间' '里程' '是否在保' '服务人员意见'
 '用户满意程度' '用户意见' '纬度' '经度' '机器所在地' '外协费用(元)' '联系人' '记录编号' '机器所在地址']


,案例编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,...,服务人员意见,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人,记录编号,机器所在地址
0,C-0002,PC360-7,113.0,土壤,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆速度慢,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,斗杆HI阀滑阀从螺纹部分脱开,P223MS,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A-0001,PC220-6,9532.0,挖土,"该车在正常工作时,全车动作突然消失,",自压减压阀卡死,"1.该机使用破碎器，作业率大约为30％，累计3000小时未更换液压油,检查液压油，发现油液污...",液压油过脏,6001ZB,判断：液压油过脏．处理：①再次清洗液压油箱，用和好的面粘净油箱中的杂质．②清洗液压油散热器，...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A-0002,PC220LC-6A(原装),16658.0,挖土,发动机高怠速时，做动作发动机冒黑烟严重，甚至会使发动机熄火，同时监控器显示E02报警，进入监...,E02报警、憋车,"1.判断后泵TVC电磁阀是否正常.断开C04插件,测量后泵TVC电磁阀的阻值15欧(标准值为...",泵控制器故障,DAB0KB,判断：由以上检测结果推断泵控制器异常.处理：更换泵控制器，故障消失，一切正常。,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A-0003,PC220-6,12340.0,破碎,"挖掘机安装破碎器后,试车时出现不击锤现象.",破碎器不击锤,"注:该故障出现前曾更换过备用阀块,但故障依然未解.1.当出现故障时,测量备用阀先导压力为32...",破碎器管路安装不齐全,7G91PJ,"判断:在破碎器的主控阀上(备用阀),下端连接先导油路,上端阀盖用堵堵死.开始使用破碎器时,先...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 读取df_combine.csv

In [23]:
df_combine = pd.read_csv('entities.csv',sep=';',encoding='utf-16')
print(df_combine.shape)
print(df_combine.columns.values)
df_combine.head()

(5740, 59)
['编号' '机型' '小时数' '工况' '故障现象' '故障内容' '检查过程' '故障原因' '故障代码' '检查结果及处置' '处理心得'
 '可能故障原因' '机号' '状态' '任务号' '报告时间' '区域' '服务人员' '是否支援' '最迟完成日期' '服务日期'
 '故障发生日期' '品牌' '工作小时(监控)' '是否停机' '结果内容' '未完成原因' '是否修复' '未修复原因'
 '故障代码翻译后的文字' '工作代码' '工作代码翻译后的文字' '是否有偿服务' '应收修理工时费(元)' '应收零件费(元)'
 '实收故障诊断工时费(元)' '实收合计(元)' '实收零件费(元)' '实收修复工时费(元)' '交机日期' '工作条件' '地面条件'
 '环境' '与客户沟通结果' '开始服务时间' '结束服务时间' '出发时间' '返回时间' '里程' '是否在保' '服务人员意见'
 '用户满意程度' '用户意见' '纬度' '经度' '机器所在地' '外协费用(元)' '联系人' '记录编号']


,编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,...,是否在保,服务人员意见,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人,记录编号
0,C-0002,PC360-7,113.0,土壤,复合动作时斗杆速度慢，挖掘无力。（新机，未出保）,斗杆速度慢,现场检查，整机工作装置单独动作时，正常。当先做大臂举升动作然后再做斗杆挖掘动作，斗杆挖掘溢流...,斗杆HI阀滑阀从螺纹部分脱开,P223MS,斗杆HI阀滑阀从中间螺纹处脱开，将该处损伤螺纹处理后重新安装故障排除。,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A-0001,PC220-6,9532.0,挖土,"该车在正常工作时,全车动作突然消失,",自压减压阀卡死,"1.该机使用破碎器，作业率大约为30％，累计3000小时未更换液压油,检查液压油，发现油液污...",液压油过脏,6001ZB,判断：液压油过脏．处理：①再次清洗液压油箱，用和好的面粘净油箱中的杂质．②清洗液压油散热器，...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A-0002,PC220LC-6A(原装),16658.0,挖土,发动机高怠速时，做动作发动机冒黑烟严重，甚至会使发动机熄火，同时监控器显示E02报警，进入监...,E02报警、憋车,"1.判断后泵TVC电磁阀是否正常.断开C04插件,测量后泵TVC电磁阀的阻值15欧(标准值为...",泵控制器故障,DAB0KB,判断：由以上检测结果推断泵控制器异常.处理：更换泵控制器，故障消失，一切正常。,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A-0003,PC220-6,12340.0,破碎,"挖掘机安装破碎器后,试车时出现不击锤现象.",破碎器不击锤,"注:该故障出现前曾更换过备用阀块,但故障依然未解.1.当出现故障时,测量备用阀先导压力为32...",破碎器管路安装不齐全,7G91PJ,"判断:在破碎器的主控阀上(备用阀),下端连接先导油路,上端阀盖用堵堵死.开始使用破碎器时,先...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 导入Neo4j

### Demo and Dragt

In [292]:
list(range(1,3))+list(range(4,7))

[1, 2, 4, 5, 6]

In [297]:
df_case.iloc[list(range(1,3))+list(range(4,7))]

,编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,处理心得,可能故障原因,机号
1,A-0001,PC220-6,9532.0,挖土,"该车在正常工作时,全车动作突然消失,",自压减压阀卡死,"1.该机使用破碎器，作业率大约为30％，累计3000小时未更换液压油,检查液压油，发现油液污...",液压油过脏,6001ZB,判断：液压油过脏．处理：①再次清洗液压油箱，用和好的面粘净油箱中的杂质．②清洗液压油散热器，...,"1、挖掘机使用破碎器时,必须按照小松保养要求进行保养（液压油和滤芯）。2、清洗液压系统时，必...",NaN,NaN
3,A-0002,PC220LC-6A(原装),16658.0,挖土,发动机高怠速时，做动作发动机冒黑烟严重，甚至会使发动机熄火，同时监控器显示E02报警，进入监...,E02报警、憋车,"1.判断后泵TVC电磁阀是否正常.断开C04插件,测量后泵TVC电磁阀的阻值15欧(标准值为...",泵控制器故障,DAB0KB,判断：由以上检测结果推断泵控制器异常.处理：更换泵控制器，故障消失，一切正常。,1、后泵TVC电磁阀线圈与壳体短路，可能为泵控器内部异常原因，如没有找到故障原因，新泵控制器...,NaN,NaN
5,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,首先分清是机械问题还是电器问题，再进一步检测，可以少走弯路。,NaN,NaN
6,A-0005,PC220-6,NaN,NaN,机器冬天温度低于零下25度，早上启动后不能回转，气温高时工作正常。,冬天早上不能回转,检查回转锁定电磁阀电阻正常，断开回转控制油路软管无液压油流出，,回转电磁阀阀芯受冻发卡,DW00M1,检查结果，回转锁定电磁阀阀芯有水及油腻造成阀芯发卡。处置结果：清洗回转电磁阀阀芯，故障排除。,1、环境温度对液压系统能否正常工作影响非常大，寒冷地区此问题比较突出，故障判断要考虑此因素。...,NaN,NaN
7,A-0006,PC200-7,1136.0,矿山,该车在行走时左边速度较慢，右边正常。测量压力及速度如下：P（左/前.后)=200KG/CM2...,左行走吸油阀故障,1、检查左行走压力补偿阀阀芯和密封——正常2、检查左行走吸油阀阀芯——阀芯正常，阀芯上的挡圈...,左行走吸油阀阀芯碳圈损坏,L5D2FA,由于该提升吸油阀（723-40-81500)的阀芯挡圈（图中第38号零件）小松不单独供应，以...,小松的压力补偿阀、吸油阀、溢流阀等阀芯上的挡圈故障较多，而小松不单独供应此类配件，在很多时候...,NaN,NaN


In [303]:
df = df_combine.iloc[[3,4,570]]
df

,编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,...,里程,是否在保,服务人员意见,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人
3,A-0003,PC220-6,12340.0,破碎,"挖掘机安装破碎器后,试车时出现不击锤现象.",破碎器不击锤,"注:该故障出现前曾更换过备用阀块,但故障依然未解.1.当出现故障时,测量备用阀先导压力为32...",破碎器管路安装不齐全,7G91PJ,"判断:在破碎器的主控阀上(备用阀),下端连接先导油路,上端阀盖用堵堵死.开始使用破碎器时,先...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,NaN,PC200-8EO,7620,采石场,NaN,回转卡,NaN,回转卡,A,测量高压共通线路,...,20.0,否,NaN,非常满意,NaN,39.145,118.453133,河北唐山市滦南县附近,0.0,李洪飞


In [78]:
from py2neo import Graph,Node,Relationship
graph = Graph('http://localhost:7474',username='neo4j',password='123123')

In [79]:
graph.delete_all()

In [324]:
df

,编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,...,里程,是否在保,服务人员意见,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人
3,A-0003,PC220-6,12340.0,破碎,"挖掘机安装破碎器后,试车时出现不击锤现象.",破碎器不击锤,"注:该故障出现前曾更换过备用阀块,但故障依然未解.1.当出现故障时,测量备用阀先导压力为32...",破碎器管路安装不齐全,7G91PJ,"判断:在破碎器的主控阀上(备用阀),下端连接先导油路,上端阀盖用堵堵死.开始使用破碎器时,先...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,NaN,PC200-8EO,7620,采石场,NaN,回转卡,NaN,回转卡,A,测量高压共通线路,...,20.0,否,NaN,非常满意,NaN,39.145,118.453133,河北唐山市滦南县附近,0.0,李洪飞


In [85]:
key_to_node

{'编号': (_921:编号 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u7f16\u53f7', value: 'A-0004'}),
 '机型': (_877:机型 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u673a\u578b', value: 'PC220-7'}),
 '故障现象': (_937:故障现象 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u6545\u969c\u73b0\u8c61', value: '\u7a7a\u8c03\u7cfb\u7edf\u4e0d\u5236\u51b7\uff0c\u65e0\u6545\u969c\u4ee3\u7801\u3002'}),
 '故障内容': (_922:故障内容 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u6545\u969c\u5185\u5bb9', value: '\u7a7a\u8c03\u7cfb\u7edf\u4e0d\u5236\u51b7'}),
 '检查过程': (_938:检查过程 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u68c0\u67e5\u8fc7\u7a0b', value: '1.\u8be5\u673a\u7a7a\u8c03\u7cfb\u7edf\u4e0d\u5236\u51b7\uff0c\u5439\u51fa\u7684\u98ce\u548c\u5916\u754c\u6c14\u6e29\u4e00\u81f4\uff0c\u5747\u4e3a20\u5ea6\u5de6\u53f3\u30022.\u8be5\u673a\u538b\u7f29\u673a\u7535\u78c1\u9600\u79bb\u5408\u5668\u4e0d\u5438\u5408\uff0c\u538b\u7f29\u673a\u4e0d\u5de5\u4f5c\u30023.\u68c0\u67e5\u7535\u78c1\u9600\u79bb\u5408\u5668\u63a7\u5236\u7ee7\u7535\u5

TODO:
1. 在创建节点前先check是否已存在
1. 实现relation.csv接口，格式为“key1,type,key2[,prop_key:prop_val]*”，如“故障原因，对应，故障代码，label=实例关系”
1. 实现properties.txt接口，格式为“key1,key2[,key3]*”，如“编号，小时数”

In [126]:
s = '你好'
print(s)

你好


In [133]:
# 解决Neo4j返还字符串的Unicode编码问题
s = str(graph.nodes.match("工况",value="破碎").first())
# s = '\u4f60\u597d'  
s = eval( "\""+s+"\"")
print(s)

(_948:工况 {label: '实例节点', type: '工况', value: '破碎'})


In [150]:
graph.nodes.match("工况",value="破 碎").first() is not None

False

In [146]:
a = graph.nodes.match("工况",value="破 碎").first() 
type(a)
a

In [140]:
graph.nodes

In [168]:
a = {'b':'test',3:'test2'}
a

{'b': 'test', 3: 'test2'}

In [191]:
dic = {label:LabelTest1,name:NameTest2}

NameError: name 'label' is not defined

In [320]:
def getPropertiesFromFile(file_path='properties.txt'):
    schema_prop = {}
    file = open(file_path)
    for s in file.readlines():
        s = s.replace('，',',') # 支持中英文逗号
        s = s.replace('\n','') # 去掉末尾的换行符
        s_sp = s.split(',')
        if (len(s_sp) < 2):
            raise(ImportError("输入字符串格式不标准，应为'key1,key2'，含义为key2字段是key1字段的属性"))
        properties = []
        for i in range(1,len(s_sp)):
            properties.append(s_sp[i])
        schema_prop[s_sp[0]] = properties
    file.close()
    notGoodKey = []
    for val in schema_prop.values():
        for v in val:
            notGoodKey.append(v)
    return schema_prop, notGoodKey
schema_prop, notGoodKey = getPropertiesFromFile()
print(notGoodKey)
schema_prop

['小时数', '工况', '处理心得', '区域', '服务人员']


{'编号': ['小时数', '工况'], '检查结果及处置': ['处理心得'], '任务号': ['区域', '服务人员']}

In [225]:
def getRelationFromString(s):
    s = s.replace('，',',') # 支持中英文逗号
    s = s.replace('：',':')
    s = s.replace('\n','') # 去掉末尾的换行符
    s_sp = s.split(',')
    if (len(s_sp) < 3):
        raise(ImportError("输入字符串格式不标准，应为'key1,relation_type,key2,property_key1=property1,property_key2=property2'"))
    dic = {}
    for i in range(3, len(s_sp)):
        dic[s_sp[i].split(':')[0]] = s_sp[i].split(':')[1]
    return {'key1':s_sp[0],'relation_type':s_sp[1],'key2':s_sp[2],'properties':dic}
#     return s_sp[0],s_sp[1],s_sp[2],dic
s = "故障原因，对应，故障代码,label:LabelTest1,name:NameTest2"
getRelationFromString(s)

{'key1': '故障原因',
 'relation_type': '对应',
 'key2': '故障代码',
 'properties': {'label': 'LabelTest1', 'name': 'NameTest2'}}

In [200]:
s = "故障原因，对应，故障代码,label:LabelTest1,name:NameTest2"
s = s.replace('，',',') # 支持中英文逗号
print(s)
s_sp = s.split(',')
dic = {}
for i in range(3, len(s_sp)):
    dic[s_sp[i].split(':')[0]] = s_sp[i].split(':')[1]
dic

故障原因,对应,故障代码,label:LabelTest1,name:NameTest2


{'label': 'LabelTest1', 'name': 'NameTest2'}

In [209]:
key1 = '故障原因'
key2 = '故障代码'
relation_type = 'test1'
properties = {'label':'LabelTest3','name':'NameTest2'}
relation = Relationship(key_to_node[key1],relation_type,key_to_node[key2])
for key in properties.keys():
    relation[key] = properties[key]
graph.create(relation)


In [212]:
s = "故障原因，test1，故障代码,label:LabelTest4,name:NameTest2"
key1, relation_type, key2, properties = getRelationFromString(s)
relation = Relationship(key_to_node[key1],relation_type,key_to_node[key2])
for key in properties.keys():
    relation[key] = properties[key]
graph.create(relation)

In [325]:
def getRelationsFromFile(file_path = 'relations.txt'):
    relations = []
    rela_file = open(file_path)
    # print(rela_file.readlines())
    for line in rela_file.readlines():
#         print(line)
#         print(getRelationFromString(line))
        relations.append(getRelationFromString(line))
    rela_file.close()
    return relations
schema_relations= getRelationsFromFile()

schema_relations

[{'key1': '故障原因',
  'relation_type': '对应',
  'key2': '故障代码',
  'properties': {'label': '实例关系', 'name': 'NameTest1'}},
 {'key1': '故障原因',
  'relation_type': '产生',
  'key2': '故障现象',
  'properties': {'label': '实例关系'}},
 {'key1': '故障现象',
  'relation_type': '处理结果',
  'key2': '检查结果及处置',
  'properties': {'label': '实例关系'}},
 {'key1': '故障现象',
  'relation_type': '对应',
  'key2': '故障内容',
  'properties': {'label': '实例关系'}},
 {'key1': '故障现象',
  'relation_type': '处理办法',
  'key2': '检查过程',
  'properties': {'label': '实例关系'}},
 {'key1': '编号',
  'relation_type': '对应',
  'key2': '机型',
  'properties': {'label': '实例关系'}},
 {'key1': '编号',
  'relation_type': '对应',
  'key2': '工况',
  'properties': {'label': '实例关系'}},
 {'key1': '编号',
  'relation_type': '对应',
  'key2': '故障现象',
  'properties': {'label': '实例关系'}},
 {'key1': '编号',
  'relation_type': '对应',
  'key2': '处理心得',
  'properties': {'label': '实例关系'}},
 {'key1': '任务号',
  'relation_type': '对应',
  'key2': '机型',
  'properties': {'label': '实例关系'}},
 {'key1': '任务号',


In [251]:
schema_prop.values()

dict_values(['小时数', '处理心得'])

In [252]:
xianxiang_node = Node("故障现象",label="概念节点",value="故障现象")
xianxiang_node['test1'] = "NodeLabelTest1"
graph.create(xianxiang_node)


In [274]:
notGoodKey = []
for val in schema_prop.values():
#     print(val)
    for v in val:
        print(v)
        notGoodKey.append(v)
notGoodKey
# [[val] for val in schema_prop.values()]

小时数
工况
处理心得


['小时数', '工况', '处理心得']

In [305]:
df

,编号,机型,小时数,工况,故障现象,故障内容,检查过程,故障原因,故障代码,检查结果及处置,...,里程,是否在保,服务人员意见,用户满意程度,用户意见,纬度,经度,机器所在地,外协费用(元),联系人
3,A-0003,PC220-6,12340.0,破碎,"挖掘机安装破碎器后,试车时出现不击锤现象.",破碎器不击锤,"注:该故障出现前曾更换过备用阀块,但故障依然未解.1.当出现故障时,测量备用阀先导压力为32...",破碎器管路安装不齐全,7G91PJ,"判断:在破碎器的主控阀上(备用阀),下端连接先导油路,上端阀盖用堵堵死.开始使用破碎器时,先...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A-0004,PC220-7,NaN,NaN,空调系统不制冷，无故障代码。,空调系统不制冷,1.该机空调系统不制冷，吹出的风和外界气温一致，均为20度左右。2.该机压缩机电磁阀离合器不...,空调控制面板损坏,8700MC,控制器控制压缩机离合器端的信号不正常，导致压缩机不能正常工作，空调系统不制冷。经更换空调控制...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570,NaN,PC200-8EO,7620,采石场,NaN,回转卡,NaN,回转卡,A,测量高压共通线路,...,20.0,否,NaN,非常满意,NaN,39.145,118.453133,河北唐山市滦南县附近,0.0,李洪飞


In [317]:
list(M.keys())+['服务人员','区域','任务号']

['编号',
 '机型',
 '机号',
 '小时数',
 '工况',
 '故障现象',
 '故障内容',
 '可能故障原因',
 '检查过程',
 '故障原因',
 '故障代码',
 '检查结果及处置',
 '处理心得',
 '服务人员',
 '区域',
 '任务号']

In [326]:
### Demo for Presentation
graph.delete_all()
bianhao_node = Node("编号",label="概念节点",value="编号")
xianxiang_node = Node("故障现象",label="概念节点",value="故障现象")
graph.create(bianhao_node)
graph.create(xianxiang_node)
for i_index in df.index:
    key_to_node = {}
    for key in list(M.keys())+['服务人员','区域','任务号']:
        print(key)
        if key in notGoodKey:
            continue
#         graph.create(Node(key))
        if df[key][i_index] == df[key][i_index]: # 判断该cell不是NaN
            nodeBefore = graph.nodes.match(key,value=df[key][i_index]).first()
            if (nodeBefore is None):
#                 print(df[key][i_index])
                key_to_node[key] = Node( key,label='实例节点',value=df[key][i_index],type=key)
                if key in schema_prop.keys():
                    for val in schema_prop[key]:
                        if df[val][i_index] == df[val][i_index]:
                            key_to_node[key][val] = df[val][i_index]
                graph.create(key_to_node[key])
            else:
                key_to_node[key] = nodeBefore

    for relation in schema_relations:
        print(relation)
        try:
            aRelation = Relationship(key_to_node[relation['key1']],relation['relation_type'],key_to_node[relation['key2']])
            properties = relation['properties']
            for key in properties.keys():
                aRelation[key] = properties[key]
            graph.create(aRelation)
        except(KeyError):
            pass
    try:
        graph.create(Relationship(key_to_node['编号'],'type',bianhao_node,label='type'))
        graph.create(Relationship(key_to_node['故障现象'],'type',xianxiang_node,label='type'))
    except(KeyError):
        pass
key_to_node

编号
机型
机号
小时数
工况
故障现象
故障内容
可能故障原因
检查过程
故障原因
故障代码
检查结果及处置
处理心得
服务人员
区域
任务号
{'key1': '故障原因', 'relation_type': '对应', 'key2': '故障代码', 'properties': {'label': '实例关系', 'name': 'NameTest1'}}
{'key1': '故障原因', 'relation_type': '产生', 'key2': '故障现象', 'properties': {'label': '实例关系'}}
{'key1': '故障现象', 'relation_type': '处理结果', 'key2': '检查结果及处置', 'properties': {'label': '实例关系'}}
{'key1': '故障现象', 'relation_type': '对应', 'key2': '故障内容', 'properties': {'label': '实例关系'}}
{'key1': '故障现象', 'relation_type': '处理办法', 'key2': '检查过程', 'properties': {'label': '实例关系'}}
{'key1': '编号', 'relation_type': '对应', 'key2': '机型', 'properties': {'label': '实例关系'}}
{'key1': '编号', 'relation_type': '对应', 'key2': '工况', 'properties': {'label': '实例关系'}}
{'key1': '编号', 'relation_type': '对应', 'key2': '故障现象', 'properties': {'label': '实例关系'}}
{'key1': '编号', 'relation_type': '对应', 'key2': '处理心得', 'properties': {'label': '实例关系'}}
{'key1': '任务号', 'relation_type': '对应', 'key2': '机型', 'properties': {'label': '实例关系'}}
{'key1': '任务号', 'relatio

{'机型': (_906:机型 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u673a\u578b', value: 'PC200-8EO'}),
 '机号': (_850:机号 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u673a\u53f7', value: 'D1078'}),
 '故障内容': (_917:故障内容 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u6545\u969c\u5185\u5bb9', value: '\u56de\u8f6c\u5361'}),
 '故障原因': (_915:故障原因 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u6545\u969c\u539f\u56e0', value: '\u56de\u8f6c\u5361'}),
 '故障代码': (_952:故障代码 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u6545\u969c\u4ee3\u7801', value: 'A'}),
 '检查结果及处置': (_926:检查结果及处置 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u68c0\u67e5\u7ed3\u679c\u53ca\u5904\u7f6e', value: '\u6d4b\u91cf\u9ad8\u538b\u5171\u901a\u7ebf\u8def'}),
 '任务号': (_860:任务号 {label: '\u5b9e\u4f8b\u8282\u70b9', type: '\u4efb\u52a1\u53f7', value: 'RW20150810093', 区域: '\u670d\u52a1-\u5510\u5c71\u56db\u533a', 服务人员: '\u738b\u5f81'})}

### publish 